# Loading the dataset via API

In [120]:
import requests
import json
import pandas as pd
import time
from datetime import datetime

def to_unix_time(dt):
    return int(time.mktime(dt.timetuple()))

def extract_current(lat, lon, key):
    getUrl = 'https://api.openweathermap.org/data/2.5/air_pollution'
    response = requests.get(getUrl, params={"lat": lat, "lon": lon, "appid": key})
    return json.loads(response.content.decode('utf-8'))

def extract_forecast(lat, lon, key):
    getUrl = 'https://api.openweathermap.org/data/2.5/air_pollution/forecast'
    response = requests.get(getUrl, params={"lat": lat, "lon": lon, "appid": key})
    return json.loads(response.content.decode('utf-8'))

def extract_history(lat, lon, start, end, key):
    getUrl = 'https://api.openweathermap.org/data/2.5/air_pollution/history'
    response = requests.get(getUrl, params={"lat": lat, "lon": lon, "start": start, "end": end, "appid": key})
    return json.loads(response.content.decode('utf-8'))

# Transform
def transform_response(response):
    # Assuming the response has data in a list of dictionaries format
    df = pd.DataFrame(response['list'])
    df['dt'] = pd.to_datetime(df['dt'], unit='s')
    # Additional transformation steps can be added here as needed
    # Extract the 'aqi' value from the 'main' dictionary
    df['aqi'] = df['main'].apply(lambda x: x['aqi'])
    # Normalize the 'components' column to create separate columns for each component
    components_df = pd.json_normalize(df['components'])
    # Concatenate the new columns to the original dataframe
    df = pd.concat([df, components_df], axis=1)
    # Drop the 'main' and 'components' columns as they are no longer needed
    df.drop(columns=['main', 'components'], inplace=True)
    return df

# The App in action: Run custom functions if valid inputs are entered

# Process
key = "49066f72f11e4c84772f0190b706335c"

# Sample latitude and longitude for demonstration
# Start time: January 1, 2021
start_dt = datetime(2021, 1, 1)
start = to_unix_time(start_dt)

# End time: December 31, 2022, 23:59:59
end_dt = datetime(2022, 12, 31, 23, 59, 59)
end = to_unix_time(end_dt)

# NYC
lat = 40.7128
lon = -74.0060
    
response_current = extract_current(lat, lon, key)
response_forecast = extract_forecast(lat, lon, key)
response_history = extract_history(lat, lon, start, end, key)

df_current = transform_response(response_current)
df_forecast = transform_response(response_forecast)
df_history = transform_response(response_history)

# Example of displaying the dataframes
display("Current Air Quality Data", df_current)
display("Forecasted Air Quality Data", df_forecast)
display("Historical Air Quality Data", df_history)


'Current Air Quality Data'

,dt,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2024-02-26 22:34:34,3,834.47,47.83,87.05,0.48,9.78,40.66,52.56,4.88


'Forecasted Air Quality Data'

,dt,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2024-02-26 22:00:00,3,620.84,22.58,72.66,15.74,12.99,32.15,40.67,3.33
1,2024-02-26 23:00:00,3,834.47,47.83,87.05,0.48,9.78,40.66,52.56,4.88
2,2024-02-27 00:00:00,3,1014.71,71.53,91.85,0.00,6.68,49.59,65.58,4.24
3,2024-02-27 01:00:00,4,1174.93,93.88,94.59,0.00,4.53,59.58,79.52,3.07
4,2024-02-27 02:00:00,4,1308.44,112.65,93.22,0.00,3.43,68.64,91.83,2.72
...,...,...,...,...,...,...,...,...,...,...
91,2024-03-01 17:00:00,2,390.53,18.77,39.76,63.66,11.68,10.40,13.61,1.69
92,2024-03-01 18:00:00,2,360.49,12.74,30.85,70.10,8.94,9.68,12.22,1.22
93,2024-03-01 19:00:00,2,347.14,8.05,23.99,78.68,7.21,11.18,13.40,0.82
94,2024-03-01 20:00:00,2,340.46,5.64,21.59,83.69,6.68,12.96,15.10,0.64


'Historical Air Quality Data'

,dt,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2020-12-31 23:00:00,2,400.54,2.29,44.55,10.64,5.96,5.03,7.76,2.53
1,2021-01-01 00:00:00,2,383.85,0.80,42.50,20.21,5.36,5.17,8.03,2.60
2,2021-01-01 01:00:00,2,400.54,0.88,44.55,20.21,5.36,6.72,10.22,3.01
3,2021-01-01 02:00:00,2,423.91,1.79,47.30,13.23,5.42,8.35,12.65,3.42
4,2021-01-01 03:00:00,2,440.60,2.91,47.98,9.12,5.13,9.36,14.02,3.55
...,...,...,...,...,...,...,...,...,...,...
17323,2022-12-31 18:00:00,3,1001.36,93.88,51.41,0.53,11.32,40.88,50.10,4.56
17324,2022-12-31 19:00:00,3,974.66,92.98,48.67,0.40,11.92,37.89,45.44,4.31
17325,2022-12-31 20:00:00,3,974.66,93.88,47.30,0.25,11.32,35.06,40.64,3.48
17326,2022-12-31 21:00:00,3,974.66,93.88,46.61,0.08,10.37,31.92,36.17,2.98


In [121]:
#From DataFrame to CSV

import os

if not os.path.exists("data"):
    os.makedirs("data")

df_history.to_csv("data/data.csv", index=False)


# TFX

In [100]:
import os

import tensorflow as tf

from tfx import v1 as tfx
import tensorflow_transform.beam as tft_beam
from google.protobuf.json_format import MessageToDict
from tensorflow_transform.tf_metadata import dataset_metadata, schema_utils
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

import tempfile
import pprint
import warnings

pp = pprint.PrettyPrinter()

# ignore tf warning messages
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore")

In [82]:
# location of the pipeline metadata store
_pipeline_root = './pipeline'

# directory of the raw data files
_data_root = './data'

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'data.csv')


In [101]:
# Declare the InteractiveContext and use a local sqlite file as the metadata store.
context = InteractiveContext(pipeline_root=_pipeline_root)


In [102]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)

# Run the component using the InteractiveContext instance
context.run(example_gen)


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [103]:
try:
    # get the artifact object
    artifact = example_gen.outputs['examples'].get()[0]
    
    # print split names and uri
    print(f'split names: {artifact.split_names}')
    print(f'artifact uri: {artifact.uri}')

# for grading since context.run() does not work outside the notebook
except IndexError:
    print("context.run() was no-op")
    examples_path = './pipeline/CsvExampleGen/examples'
    dir_id = os.listdir(examples_path)[0]
    artifact_uri = f'{examples_path}/{dir_id}'

else:
    artifact_uri = artifact.uri
    

split names: ["train", "eval"]
artifact uri: ./pipeline\CsvExampleGen\examples\1


In [104]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(artifact_uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")


In [105]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs['examples'])


# Run the component
context.run(statistics_gen)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [106]:
# Plot the statistics generated
context.show(statistics_gen.outputs['statistics'])


In [107]:
# Instantiate SchemaGen with the output statistics from the StatisticsGen
schema_gen = tfx.components.SchemaGen(statistics_gen.outputs['statistics'])
    
# Run the component
context.run(schema_gen)


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [108]:
# Visualize the output
context.show(schema_gen.outputs['schema'])


,Type,Presence,Valency,Domain
Feature name,,,,
'aqi',INT,required,,-
'co',FLOAT,required,,-
'dt',BYTES,required,,-
'nh3',FLOAT,required,,-
'no',FLOAT,required,,-
'no2',FLOAT,required,,-
'o3',FLOAT,required,,-
'pm10',FLOAT,required,,-
'pm2_5',FLOAT,required,,-


In [109]:
# Instantiate ExampleValidator with the statistics and schema from the previous steps
example_validator = tfx.components.ExampleValidator(statistics=statistics_gen.outputs['statistics'],
                                  schema=schema_gen.outputs['schema'])
    
# Run the component
context.run(example_validator)


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [110]:
# Visualize the output
context.show(example_validator.outputs['anomalies'])


In [111]:
# Set the constants module filename
_traffic_constants_module_file = 'traffic_constants.py'


In [112]:
%%writefile {_traffic_constants_module_file}

# Features to be scaled to the z-score
DENSE_FLOAT_FEATURE_KEYS = ['co', 'nh3', 'no', 'no2', 'o3', 'pm10', 'pm2_5', 'so2']


def transformed_name(key):
    return key + '_xf'


Writing traffic_constants.py


In [113]:
# Set the transform module filename
_traffic_transform_module_file = 'traffic_transform.py'


In [114]:
%%writefile {_traffic_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import traffic_constants

# Unpack the contents of the constants module
_DENSE_FLOAT_FEATURE_KEYS = traffic_constants.DENSE_FLOAT_FEATURE_KEYS
_transformed_name = traffic_constants.transformed_name

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
    inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
    Map from string feature key to transformed feature operations.
    """
    outputs = {}


    # Scale these features to the z-score.
    for key in _DENSE_FLOAT_FEATURE_KEYS:
        # Scale these features to the z-score.
        outputs[_transformed_name(key)] = tft.scale_to_z_score(inputs[key])


    return outputs


Writing traffic_transform.py


In [115]:
# Instantiate the Transform component
transform = tfx.components.Transform(examples=example_gen.outputs['examples'],
                                     schema=schema_gen.outputs['schema'],
                                     module_file=os.path.abspath(_traffic_transform_module_file)
                                    )

# Run the component.
context.run(transform, enable_cache=False)


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Preprocessing

In [124]:
csv_url = 'data/data.csv'
df_history = pd.read_csv(csv_url)
df_history

,dt,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2020-12-31 23:00:00,2,400.54,2.29,44.55,10.64,5.96,5.03,7.76,2.53
1,2021-01-01 00:00:00,2,383.85,0.80,42.50,20.21,5.36,5.17,8.03,2.60
2,2021-01-01 01:00:00,2,400.54,0.88,44.55,20.21,5.36,6.72,10.22,3.01
3,2021-01-01 02:00:00,2,423.91,1.79,47.30,13.23,5.42,8.35,12.65,3.42
4,2021-01-01 03:00:00,2,440.60,2.91,47.98,9.12,5.13,9.36,14.02,3.55
...,...,...,...,...,...,...,...,...,...,...
17323,2022-12-31 18:00:00,3,1001.36,93.88,51.41,0.53,11.32,40.88,50.10,4.56
17324,2022-12-31 19:00:00,3,974.66,92.98,48.67,0.40,11.92,37.89,45.44,4.31
17325,2022-12-31 20:00:00,3,974.66,93.88,47.30,0.25,11.32,35.06,40.64,3.48
17326,2022-12-31 21:00:00,3,974.66,93.88,46.61,0.08,10.37,31.92,36.17,2.98


In [125]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import pandas as pd

def preprocess_data(df):
    # Normalize the pollutant columns (except 'aqi') using MinMaxScaler
    pollutants = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']
    scaler = MinMaxScaler()
    for pollutant in pollutants:
        df[pollutant] = scaler.fit_transform(df[[pollutant]])  # No need to reshape as we are using DataFrame slicing

    # Extract 'year', 'month', 'day', 'hour', and 'weekday' from the 'dt' column
    df['year'] = df['dt'].dt.year
    df['month'] = df['dt'].dt.month
    df['day'] = df['dt'].dt.day
    df['hour'] = df['dt'].dt.hour
    df['weekday'] = df['dt'].dt.day_name()  # Add the weekday names

    # Perform one-hot encoding using pd.get_dummies()
    df = pd.get_dummies(df, columns=['year', 'month', 'day', 'hour', 'weekday'])

    return df

csv_url = 'data/data.csv'
df_history = pd.read_csv(csv_url, parse_dates=['dt'])

# Apply preprocessing to the DataFrame
df_history_preprocessed = preprocess_data(df_history)

# Display the preprocessed DataFrame
display("Preprocessed Historical Air Quality Data", df_history_preprocessed)
df_history_preprocessed.drop(columns=['dt'], inplace=True)


'Preprocessed Historical Air Quality Data'

,dt,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3,...,hour_21,hour_22,hour_23,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,2020-12-31 23:00:00,2,0.049094,0.003638,0.165375,0.026189,0.049175,0.013716,0.018125,0.038269,...,0,0,1,0,0,0,0,1,0,0
1,2021-01-01 00:00:00,2,0.045402,0.001271,0.157349,0.049745,0.043178,0.014140,0.018814,0.039332,...,0,0,0,1,0,0,0,0,0,0
2,2021-01-01 01:00:00,2,0.049094,0.001398,0.165375,0.049745,0.043178,0.018834,0.024405,0.045558,...,0,0,0,1,0,0,0,0,0,0
3,2021-01-01 02:00:00,2,0.054263,0.002844,0.176141,0.032565,0.043778,0.023769,0.030609,0.051784,...,0,0,0,1,0,0,0,0,0,0
4,2021-01-01 03:00:00,2,0.057955,0.004623,0.178804,0.022448,0.040880,0.026827,0.034106,0.053759,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17323,2022-12-31 18:00:00,3,0.181985,0.149151,0.192232,0.001305,0.102749,0.122267,0.126213,0.069096,...,0,0,0,0,0,1,0,0,0,0
17324,2022-12-31 19:00:00,3,0.176080,0.147721,0.181505,0.000985,0.108746,0.113214,0.114316,0.065300,...,0,0,0,0,0,1,0,0,0,0
17325,2022-12-31 20:00:00,3,0.176080,0.149151,0.176141,0.000615,0.102749,0.104645,0.102063,0.052696,...,0,0,0,0,0,1,0,0,0,0
17326,2022-12-31 21:00:00,3,0.176080,0.149151,0.173440,0.000197,0.093253,0.095137,0.090651,0.045103,...,1,0,0,0,0,1,0,0,0,0


In [126]:
#From DataFrame to CSV
df_history_preprocessed.to_csv("data/data_preprocess.csv", index=False) 

# MLflow

https://medium.com/walmartglobaltech/model-and-data-versioning-an-introduction-to-mlflow-and-dvc-260347cd0f6e

In [130]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score  # Import necessary metrics
import pandas as pd

# Replace 'csv_url' with the actual URL of your CSV data
csv_url = 'data/data_preprocess.csv'
data = pd.read_csv(csv_url)

# Assuming the last column is the target variable
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

# Splitting the dataset into training and testing sets
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


## Part 1

In [131]:

# Set the experiment name
mlflow.set_experiment("Random Forest Model Tracking")

with mlflow.start_run(run_name="Random Forest Experiment"):
    # Initialize and train the Random Forest model
    params = {"n_estimators": 100, "max_depth": 5, "random_state": 42}
    model = RandomForestClassifier(**params)
    model.fit(train_x, train_y)
    
    # Predict and evaluate the model using the metrics
    predicted_y = model.predict(test_x)
    f1 = f1_score(test_y, predicted_y, average='weighted')  # Calculate F1 score
    precision = precision_score(test_y, predicted_y, average='weighted')  # Calculate precision
    recall = recall_score(test_y, predicted_y, average='weighted')  # Calculate recall
    accuracy = accuracy_score(test_y, predicted_y)  # Calculate accuracy
    
    # Log parameters and all metrics
    mlflow.log_params(params)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("accuracy", accuracy)
    
    # Log the Random Forest model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="random_forest_model",
        registered_model_name="RandomForestModel"
    )

Successfully registered model 'RandomForestModel'.
Created version '1' of model 'RandomForestModel'.


In [132]:

# Set the experiment name
mlflow.set_experiment("Random Forest Model Tracking")

with mlflow.start_run(run_name="Random Forest Experiment"):
    # Initialize and train the Random Forest model
    params = {"n_estimators": 90, "max_depth": 6, "random_state": 42}
    model = RandomForestClassifier(**params)
    model.fit(train_x, train_y)
    
    # Predict and evaluate the model using the metrics
    predicted_y = model.predict(test_x)
    f1 = f1_score(test_y, predicted_y, average='weighted')  # Calculate F1 score
    precision = precision_score(test_y, predicted_y, average='weighted')  # Calculate precision
    recall = recall_score(test_y, predicted_y, average='weighted')  # Calculate recall
    accuracy = accuracy_score(test_y, predicted_y)  # Calculate accuracy
    
    # Log parameters and all metrics
    mlflow.log_params(params)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("accuracy", accuracy)
    
    # Log the Random Forest model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="random_forest_model",
        registered_model_name="RandomForestModel"
    )

Registered model 'RandomForestModel' already exists. Creating a new version of this model...
Created version '2' of model 'RandomForestModel'.


In [133]:
from sklearn.svm import SVC

# Set the experiment name
mlflow.set_experiment("SVM Model Tracking")

with mlflow.start_run(run_name="SVM Experiment"):
    # Initialize and train the SVM model
    params = {"C": 1.0, "kernel": "rbf", "random_state": 42}  # Default parameters for SVC, adjust as needed
    model = SVC(**params)
    model.fit(train_x, train_y)
    
    # Predict and evaluate the model using the metrics
    predicted_y = model.predict(test_x)
    f1 = f1_score(test_y, predicted_y, average='weighted')  # Calculate F1 score
    precision = precision_score(test_y, predicted_y, average='weighted', zero_division=0)  # Calculate precision
    recall = recall_score(test_y, predicted_y, average='weighted', zero_division=0)  # Calculate recall
    accuracy = accuracy_score(test_y, predicted_y)  # Calculate accuracy
    
    # Log parameters and all metrics
    mlflow.log_params(params)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("accuracy", accuracy)
    
    # Log the SVM model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="svm_model",
        registered_model_name="SVMModel"
    )


Successfully registered model 'SVMModel'.
Created version '1' of model 'SVMModel'.


## Part 2

In [134]:

from mlflow.tracking import MlflowClient
client = MlflowClient()


client.transition_model_version_stage(
    name="SVMModel",
    version=1,
    stage="Staging"
)
client.transition_model_version_stage(
    name="RandomForestModel",
    version=1,
    stage="Staging"
)

# Promoting a model to Production stage
# Replace <model> with your model name. Assuming version 2 is the one you want to promote
client.transition_model_version_stage(
    name="SVMModel",
    version=1,
    stage="Production"
)



# Archive an old version of model
# Replace <model> with your model name. Assuming version 3 is the one you want to archive
client.transition_model_version_stage(
    name="SVMModel",
    version=1,
    stage="Archived"
)

# Promoting a model to Production stage
# Replace <model> with your model name. Assuming version 2 is the one you want to promote
client.transition_model_version_stage(
    name="RandomForestModel",
    version=2,
    stage="Production"
)

<ModelVersion: aliases=[], creation_timestamp=1708987041788, current_stage='Production', description=None, last_updated_timestamp=1708987074713, name='RandomForestModel', run_id='c10616d1b03e4fce8f818744fe0815d4', run_link=None, source='file:///C:/Users/julie/Desktop/Nouveau%20dossier/tfx/mlruns/626554607180509191/c10616d1b03e4fce8f818744fe0815d4/artifacts/random_forest_model', status='READY', status_message=None, tags={}, user_id=None, version=2>

## Part 3

In [135]:
# Loading a Specific Version of the Model

import mlflow.pyfunc

model_name = "RandomForestModel"
stage = 'Staging'  # Assuming you have a version of the model in the 'Staging' stage
model_uri = f"models:/{model_name}/{stage}"
model = mlflow.pyfunc.load_model(model_uri=model_uri)

# Load a sample of data for prediction
# Ensure 'data' is preprocessed similarly to the training data
predictions = model.predict(test_x)  # Assuming test_x is available
print(predictions)


[1 1 4 ... 1 1 2]


In [136]:
# Loading a Specific Type of Models like sklearn

import mlflow.sklearn

model_path = "models:/RandomForestModel/1"  # Replace '1' with your specific version number
model = mlflow.sklearn.load_model(model_path)

# Use the model for predictions
predictions = model.predict(test_x)
print(predictions)


[1 1 4 ... 1 1 2]


In [137]:
# Loading the Latest Production Version of the Model

import mlflow.pyfunc

model_name = "RandomForestModel"
stage = 'Production'
model_uri = f"models:/{model_name}/{stage}"
model = mlflow.pyfunc.load_model(model_uri=model_uri)

# Predict with the latest production model
predictions = model.predict(test_x)
print(predictions)

[1 1 4 ... 1 1 2]
